In [1]:
import pathlib
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import tensorflow_addons as tfa
import random

from tensorflow import keras
from tensorflow.keras import layers, Sequential
from tensorflow.keras.applications import EfficientNetB3
from functools import partial
from albumentations import (Compose, Transpose, HorizontalFlip,
                            VerticalFlip, RandomResizedCrop, RandomRotate90, RandomBrightnessContrast,HueSaturationValue)

In [2]:
hyper_params={
    "img_size": 512,
    "batch_size": 8,
    "val_ratio": 0.2,
    "buffer_size": 1000,
    "num_classes": 5,
    "fold": 5,
    "seed": 2021,

    "epochs": 10,
    "init_lr": float(1e-3)
}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.compat.v1.set_random_seed(seed)
    
seed_everything(hyper_params["seed"])

In [4]:
class generate_data():
    def __init__(self,
                 image,
                 label,
                 img_size=224):
        self.image = image
        self.label = label
        self.img_size = img_size
        self.img_num = len(image)
        self.AUTOTUNE = tf.data.experimental.AUTOTUNE

    def _create_dataset(self):
        list_ds = tf.data.Dataset.from_tensor_slices(
            (self.image, self.label))

        list_ds = list_ds.shuffle(
            self.img_num, reshuffle_each_iteration=False)
        
        return list_ds

    def _resize_img(self, file_path, label):
        img = tf.io.read_file(file_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [self.img_size, self.img_size])

        return img, label
    
    def get_data(self):
        ds = self._create_dataset()
        ds = ds.map(self._resize_img, num_parallel_calls=self.AUTOTUNE)

        return ds

In [5]:
augmentate = Compose([
    RandomResizedCrop(hyper_params["img_size"], hyper_params["img_size"]),
    Transpose(p=0.6),
    HorizontalFlip(p=0.6),
    VerticalFlip(p=0.6),
    RandomRotate90(p=0.6),
    RandomBrightnessContrast(brightness_limit=(-0.1, 0.1),
                             contrast_limit=(-0.1, 0.1), p=0.6),
    HueSaturationValue(hue_shift_limit=0.2,
                       sat_shift_limit=0.2, val_shift_limit=0.2, p=0.6)
])


class augmentate_data():
    def __init__(self, ds, img_size):
        self.ds = ds
        self.input_shape=(img_size, img_size, 3)

    def _aug_fn(self, image):
        data = {"image": image}
        aug_data = augmentate(**data)
        aug_img = aug_data["image"]

        return aug_img

    def _process_img(self, image, label):
        aug_img = tf.numpy_function(
            func=self._aug_fn, inp=[image], Tout=tf.float32)

        return aug_img, label

    def _set_shapes(self, img, label):
        img.set_shape((self.input_shape))
        label.set_shape([])

        return img, label

    def data_aug(self):
        ds = self.ds.map(self._process_img,
                         num_parallel_calls=tf.data.experimental.AUTOTUNE)

        ds = self.ds.map(self._set_shapes,
                         num_parallel_calls=tf.data.experimental.AUTOTUNE)
        return ds

In [6]:
def efficientnetB3(num_classes):
    model = Sequential()
    pretrained_b3 = EfficientNetB3(include_top=False, weights="imagenet",
                                   input_shape=(hyper_params["img_size"], hyper_params["img_size"], 3), drop_connect_rate=0.4)
    model.add(pretrained_b3)
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [7]:
"""
@tf.function
def train_step(X, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

def train():
    loss_value = train_step(X, y)
    
    if step%100==0:
        print("Training loss (for one batch) at step %d: %.4f" % (step, float(loss_value)))
    
    train_acc = train_acc_metric.result()
"""

'\n@tf.function\ndef train_step(X, y):\n    with tf.GradientTape() as tape:\n        logits = model(x, training=True)\n        loss_value = loss_fn(y, logits)\n    grads = tape.gradient(loss_value, model.trainable_weights)\n    optimizer.apply_gradients(zip(grads, model.trainable_weights))\n    train_acc_metric.update_state(y, logits)\n    return loss_value\n\ndef train():\n    loss_value = train_step(X, y)\n    \n    if step%100==0:\n        print("Training loss (for one batch) at step %d: %.4f" % (step, float(loss_value)))\n    \n    train_acc = train_acc_metric.result()\n'

In [8]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Read CSV file
train_df = pd.read_csv("../input/split-train-data-to-5-fold/train_df_fold_3.csv")
val_df = pd.read_csv("../input/split-train-data-to-5-fold/val_df_fold_3.csv")

train_ds = generate_data(
    train_df["file_path"], train_df["label"], hyper_params["img_size"]).get_data()
val_ds = generate_data(val_df["file_path"],
                       val_df["label"], hyper_params["img_size"]).get_data()

train_ds = augmentate_data(train_ds, hyper_params["img_size"]).data_aug()

train_ds = train_ds.shuffle(buffer_size=hyper_params["buffer_size"]).batch(
    hyper_params["batch_size"]).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.shuffle(buffer_size=hyper_params["buffer_size"]).batch(
    hyper_params["batch_size"]).prefetch(buffer_size=AUTOTUNE)
    
# Build model
model = efficientnetB3(hyper_params["num_classes"])
model.summary()
    
    
# Train
early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=5, restore_best_weights=True)

decay_step = int((len(train_ds)/hyper_params["batch_size"])*hyper_params["epochs"])

CosDecay = tf.keras.experimental.CosineDecay(hyper_params["init_lr"], decay_steps=decay_step, alpha=1e-3)
    
Adam = keras.optimizers.Adam(learning_rate=CosDecay)

Lookahead = tfa.optimizers.Lookahead(Adam)
    
model.compile(optimizer=Lookahead,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=False),
              metrics=["accuracy"])
    

model.fit(train_ds, validation_data=val_ds, epochs=hyper_params["epochs"], callbacks=[early_stop])

model.save("./Lookahead_Adam_CosDecay_Fold_3.h5")

43941888/43941136 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb3 (Functional)  (None, 16, 16, 1536)      10783535  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 7685      
Total params: 10,791,220
Trainable params: 10,703,917
Non-trainable params: 87,303
_________________________________________________________________
Epoch 1/10
2140/2140 [==============================] - 1332s 622ms/step - loss: 0.5579 - accuracy: 0.8076 - val_loss: 0.3481 - val_accuracy: 0.8822
Epoch 2/10
2140/2140 [=